# 01 – Exploración del subconjunto de OpenFoodFacts (OFF)

En este notebook se explora el fichero procesado:

- Entrada: `data_processed/openfoodfacts/openfoodfacts_subset.parquet`
- Origen: volcado original `en.openfoodfacts.org.products.csv` (~11 GB)
- Número aproximado de filas: ~4,2 millones
- Número de columnas: 23 (identificadores, países, categorías PNNS,
  nutrientes por 100 g, grupo NOVA y fechas).

**Objetivos de este notebook**

1. Verificar la estructura básica (filas, columnas, tipos).
2. Explorar países y categorías (`pnns_groups_1` y `pnns_groups_2`).
3. Evaluar la completitud de los nutrientes clave (energía, azúcares,
   grasas saturadas, sodio, proteínas).
4. Sentar la base para definir filtros posteriores (UE-27, periodo temporal,
   criterios de calidad de datos).

In [1]:
from pathlib import Path
import duckdb
import pandas as pd

# Carpeta raíz del proyecto (subimos desde notebooks/exploratory)
ROOT_DIR = Path("..").resolve().parent

# Carpeta con datos procesados de OFF
DATA_PROCESSED = ROOT_DIR / "data_processed" / "openfoodfacts"

# Fichero Parquet creado en el pre-ETL
PARQUET_PATH = DATA_PROCESSED / "openfoodfacts_subset.parquet"

ROOT_DIR, PARQUET_PATH, PARQUET_PATH.exists()

(WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm'),
 WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm/data_processed/openfoodfacts/openfoodfacts_subset.parquet'),
 True)

In [2]:
# Conexión en memoria para consultas
con = duckdb.connect(database=":memory:")

preview = con.execute(f"""
    SELECT *
    FROM read_parquet('{PARQUET_PATH}')
    LIMIT 5
""").fetchdf()

preview

,code,product_name,brands,countries,countries_tags,categories,categories_tags,pnns_groups_1,pnns_groups_2,quantity,...,saturated-fat_100g,sodium_100g,salt_100g,fat_100g,carbohydrates_100g,fiber_100g,proteins_100g,nova_group,created_datetime,last_modified_datetime
0,00000002,None,None,en:Germany,en:germany,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2025-10-19 03:13:03-05:00,2025-10-19 03:13:06-05:00
1,00000003,None,None,en:France,en:france,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2025-07-14 04:29:48-05:00,2025-07-14 04:29:49-05:00
2,00000004,None,None,en:france,en:france,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2025-11-19 05:35:27-05:00,2025-11-19 05:55:02-05:00
3,00000005,None,None,en:France,en:france,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2025-08-04 08:27:01-05:00,2025-08-04 08:27:03-05:00
4,00000006,None,None,en:Germany,en:germany,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2025-10-11 15:02:55-05:00,2025-10-11 16:42:10-05:00


In [3]:
# Número de filas
n_rows = con.execute(f"""
    SELECT COUNT(*) AS n_filas
    FROM read_parquet('{PARQUET_PATH}')
""").fetchdf()

# DataFrame vacío para ver estructura (0 filas, todas las columnas)
schema_df = con.execute(f"""
    SELECT *
    FROM read_parquet('{PARQUET_PATH}')
    LIMIT 0
""").fetchdf()

n_filas = int(n_rows["n_filas"][0])
n_columnas = schema_df.shape[1]

n_filas, n_columnas

(4210261, 23)

In [4]:
schema_df.dtypes

code                                              object
product_name                                      object
brands                                            object
countries                                         object
countries_tags                                    object
categories                                        object
categories_tags                                   object
pnns_groups_1                                     object
pnns_groups_2                                     object
quantity                                          object
serving_size                                      object
energy_100g                                      float64
sugars_100g                                      float64
saturated-fat_100g                               float64
sodium_100g                                      float64
salt_100g                                        float64
fat_100g                                         float64
carbohydrates_100g             

In [5]:
top_countries = con.execute(f"""
    SELECT countries, COUNT(*) AS n
    FROM read_parquet('{PARQUET_PATH}')
    GROUP BY countries
    ORDER BY n DESC
    LIMIT 20
""").fetchdf()

top_countries

,countries,n
0,France,506787
1,United States,357778
2,en:fr,331068
3,en:us,217171
4,en:france,186387
5,en:it,173994
6,en:United States,168835
7,en:germany,141285
8,en:es,140929
9,España,109177


In [6]:
pnns_1 = con.execute(f"""
    SELECT pnns_groups_1, COUNT(*) AS n
    FROM read_parquet('{PARQUET_PATH}')
    GROUP BY pnns_groups_1
    ORDER BY n DESC
""").fetchdf()

pnns_2 = con.execute(f"""
    SELECT pnns_groups_2, COUNT(*) AS n
    FROM read_parquet('{PARQUET_PATH}')
    GROUP BY pnns_groups_2
    ORDER BY n DESC
""").fetchdf()

pnns_1.head(20), pnns_2.head(20)

(              pnns_groups_1        n
 0                   unknown  2782034
 1             Sugary snacks   282347
 2            Fish Meat Eggs   187637
 3      Cereals and potatoes   174510
 4   Milk and dairy products   168444
 5                 Beverages   133205
 6            Fat and sauces   121329
 7           Composite foods   101500
 8     Fruits and vegetables    98080
 9              Salty snacks    79787
 10                     None    44860
 11      Alcoholic beverages    36528,
             pnns_groups_2        n
 0                 unknown  2782034
 1                  Sweets   127658
 2      Biscuits and cakes   111442
 3    Dressings and sauces    85862
 4                 Cereals    78331
 5                  Cheese    76863
 6          One-dish meals    69770
 7          Processed meat    66477
 8                    Meat    58686
 9         Milk and yogurt    57979
 10             Vegetables    49608
 11       Fish and seafood    48601
 12                  Bread    46039
 

In [7]:
sample_df = con.execute(f"""
    SELECT *
    FROM read_parquet('{PARQUET_PATH}')
    LIMIT 100000
""").fetchdf()

na_ratio = sample_df.isna().mean().sort_values(ascending=False)
na_ratio

quantity                  0.72654
nova_group                0.49022
categories_tags           0.45430
categories                0.45430
fiber_100g                0.39977
brands                    0.39491
serving_size              0.37332
saturated-fat_100g        0.29285
salt_100g                 0.28120
sodium_100g               0.28120
sugars_100g               0.24537
fat_100g                  0.20540
proteins_100g             0.20478
carbohydrates_100g        0.20228
energy_100g               0.19801
product_name              0.05431
pnns_groups_2             0.00679
pnns_groups_1             0.00679
countries_tags            0.00520
countries                 0.00519
code                      0.00000
created_datetime          0.00000
last_modified_datetime    0.00000
dtype: float64

In [8]:
nutr_cols = [
    "energy_100g",
    "sugars_100g",
    "saturated-fat_100g",
    "sodium_100g",
    "fat_100g",
    "carbohydrates_100g",
    "fiber_100g",
    "proteins_100g",
]

na_nutrients = na_ratio[nutr_cols]
na_nutrients

energy_100g           0.19801
sugars_100g           0.24537
saturated-fat_100g    0.29285
sodium_100g           0.28120
fat_100g              0.20540
carbohydrates_100g    0.20228
fiber_100g            0.39977
proteins_100g         0.20478
dtype: float64

In [9]:
valid_stats = con.execute(f"""
    SELECT
        COUNT(*) AS n_total,
        -- los 4 nutrientes presentes
        SUM(
            (energy_100g IS NOT NULL)::INT
          * (sugars_100g IS NOT NULL)::INT
          * ("saturated-fat_100g" IS NOT NULL)::INT
          * (sodium_100g IS NOT NULL)::INT
        ) AS n_4_nutrientes,
        -- al menos 3 de los 4 presentes
        SUM(
            (
              (energy_100g IS NOT NULL)::INT
            + (sugars_100g IS NOT NULL)::INT
            + ("saturated-fat_100g" IS NOT NULL)::INT
            + (sodium_100g IS NOT NULL)::INT
            ) >= 3
        ) AS n_al_menos_3
    FROM read_parquet('{PARQUET_PATH}')
""").fetchdf()

valid_stats

,n_total,n_4_nutrientes,n_al_menos_3
0,4210261,2466120.0,2871200.0


In [10]:
n_total = valid_stats["n_total"][0]
n_4 = valid_stats["n_4_nutrientes"][0]
n_3 = valid_stats["n_al_menos_3"][0]

porc_4 = n_4 / n_total
porc_3 = n_3 / n_total

n_total, n_4, porc_4, n_3, porc_3

(np.int64(4210261),
 np.float64(2466120.0),
 np.float64(0.5857404089675201),
 np.float64(2871200.0),
 np.float64(0.6819529715616205))

## 5. Conclusiones de la exploración de OpenFoodFacts

A partir del volcado original de OpenFoodFacts (~11 GB) se generó un subconjunto en formato Parquet con 4.210.261 productos y 23 variables, que integra identificadores básicos, países declarados, grupos PNNS, nutrientes por 100 g, grupo NOVA y fechas de creación/modificación.

La inspección inicial muestra que la variable `countries` combina nombres de países escritos de forma libre (`France`, `United States`, `España`) con etiquetas codificadas (`en:fr`, `en:germany`, `en:spain`). Esto confirma la necesidad de un proceso posterior de normalización de países y de filtrado a los Estados miembros de la Unión Europea.

Los grupos de alimentos PNNS (`pnns_groups_1` y `pnns_groups_2`) están presentes para una parte relevante de los registros, pero existe una categoría masiva `unknown` (~2,78 millones de productos) que deberá tratarse de manera específica en el ETL, ya sea excluyéndola del análisis principal o analizándola como grupo residual.

En cuanto a la calidad de los nutrientes, el análisis de una muestra de 100.000 filas indica tasas de ausencia importantes: alrededor del 20–30 % de valores nulos en energía, azúcares, grasas saturadas y sodio, y hasta un 40 % en fibra. Sin embargo, al considerar la base completa, aproximadamente el 58,6 % de los productos dispone de los cuatro nutrientes clave, y el 68,2 % tiene al menos tres de ellos informados. Este resultado es suficiente para construir indicadores nutricionales robustos si se restringe el análisis a dicho subconjunto “bien rellenado”.

En conjunto, este notebook establece la estructura y las limitaciones del dataset de OFF, y proporciona los criterios de partida para el siguiente paso del flujo de trabajo: definir el ETL específico para obtener un subconjunto coherente con la UE-27 y con un nivel mínimo de completitud nutricional.

In [11]:
resumen_off = pd.DataFrame(
    {
        "indicador": [
            "Filas totales en Parquet",
            "Columnas totales",
            "Productos con 4 nutrientes clave",
            "% con 4 nutrientes clave",
            "Productos con ≥3 nutrientes clave",
            "% con ≥3 nutrientes clave",
        ],
        "valor": [
            int(n_total),
            int(n_columnas),
            int(n_4),
            round(float(porc_4) * 100, 2),
            int(n_3),
            round(float(porc_3) * 100, 2),
        ],
    }
)

resumen_off

,indicador,valor
0,Filas totales en Parquet,4210261.00
1,Columnas totales,23.00
2,Productos con 4 nutrientes clave,2466120.00
3,% con 4 nutrientes clave,58.57
4,Productos con ≥3 nutrientes clave,2871200.00
5,% con ≥3 nutrientes clave,68.20
